In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,mapping,LineString, Point
import math
import tqdm
import sqlite3
from calculo_predial import *
from Valor_catastral_construccion import *
from Valor_catastral_terreno import *
from Herramientas import *

/home/hector/anaconda3/envs/Infis/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_17440/3317965230.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/mig

In [3]:
# Inputs

# Modelo IA
path_modelo_casas = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Ixtapan_de_la_sal/Ixtapan_Sal_casas.shp'
# Manzanas del municipio
path_manzanas     = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Ixtapan_de_la_sal/Manzana_Ixtapan_Sal.shp'
# Cruce de valores unitarios vs Manzana
#path_vu_and_mz    = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Valores_Unitarios_con_Manzanas_Naucalpan.shp'
# Valores unitarios
path_sql          ='Data/Valores_Unitarios_2022_v2'
# shape 15m 
path_info = '/home/hector/Documentos/Infis/Geo/Data/Shapes/15m/15m.shp'
# path de chinchetas
path_chinchetas = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Ixtapan_de_la_sal/chinchetas_limpieza1/Centro_chinchetas_limpieza1.shp'
# ruta a dnue
path_dnue = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Dnue/DENUE_EDOMEX_2022.shp'
# Lista de afirmaciones en minisculas
afirmativo = np.array(['si','1',1,'yes','claro','obvio','of course','sip','sipi','neta','ok'])
# Municipio a procesar
municipio = 'Ixtapan_de_la_sal'

## Proceso 0

In [4]:
# Importar valores de SQL

conexion = sqlite3.connect(path_sql)
df_area = pd.read_sql(f'''
SELECT Zona,Manzanas,COD,Tipo,Frente,Fondo,Area as Area_AH,[Val m2],
        Pseudo_clave_cat, Municipio
FROM area_homogenea	
WHERE Municipio = '{municipio}'
    ''', conexion)
df_banda = pd.read_sql(f'''
SELECT [Tipo calle], [Nombre Calle], [Zona], [Manzanas], COD,
       [Val m2], Pseudo_clave_cat, Municipio
FROM banda_valores
WHERE Municipio = '{municipio}'
''', conexion)

conexion.close()

In [5]:
# Corregir formato del val m2 en ambas
df_area['Val m2'] = df_area['Val m2'].map(lambda x: corregir_formato(str(x)))
df_banda['Val m2'] = df_banda['Val m2'].map(lambda x: corregir_formato(str(x)))

In [6]:
# Visualizar que tenemos el municipio adecuado
df_area['Municipio'].value_counts(), df_banda['Municipio'].value_counts()

(Municipio
 Ixtapan_de_la_sal    794
 Name: count, dtype: int64,
 Municipio
 Ixtapan_de_la_sal    220
 Name: count, dtype: int64)

In [7]:
# Borramos duplicados
print('Shape inicial de Area_H ', df_area.shape)
print('Shape inicial de bandaV ', df_banda.shape)
df_area.drop_duplicates(subset=['Zona', 'Manzanas', 'COD', 'Tipo', 'Frente', 'Fondo', 'Area_AH',
       'Val m2', 'Pseudo_clave_cat'],
       inplace=True)
df_banda.drop_duplicates(subset=['Tipo calle', 'Nombre Calle', 'Zona', 'Manzanas', 'COD', 'Val m2',
       'Pseudo_clave_cat'], inplace=True)
print('Shape Final de Area_H ', df_area.shape)
print('Shape Final de bandaV ', df_banda.shape)

Shape inicial de Area_H  (794, 10)
Shape inicial de bandaV  (220, 8)
Shape Final de Area_H  (397, 10)
Shape Final de bandaV  (110, 8)


In [8]:
# Cargamos el shape de manzanas para analisis
shape = gpd.read_file(path_manzanas)
shape=shape.to_crs("3857")
shape['key'] = shape['cve_cat'].map(lambda x: str(x)[:8])
shape['Lon'] = shape['geometry'].centroid.x
shape['Lat'] = shape['geometry'].centroid.y
shape.shape

(661, 8)

In [9]:
print(f'Shape de Area Homo= {df_area.shape}')
print(f'Shape de Banda    = {df_banda.shape}')
print(f'Shape de Manzanas = {shape.shape}')

Shape de Area Homo= (397, 10)
Shape de Banda    = (110, 8)
Shape de Manzanas = (661, 8)


In [10]:
len(df_banda[~df_banda["Manzanas"].isin(df_area["Manzanas"])]["Manzanas"].unique())

7

In [11]:
# Cruce area_h vs Mzs
cruce1 = df_area.merge(shape, 
                       left_on='Pseudo_clave_cat', right_on='key', 
                       how='outer', indicator=True, 
                       suffixes=('_AreaH', '_Manz'))
cruce1['Tipo_cruce1'] = cruce1['_merge'].map({'both':'AreaH_&_Manzana', 'left_only':'AreaH', 'right_only':'Manzanas' })
cruce1.drop(columns=['_merge'], inplace=True)
cruce1.shape

(685, 19)

In [12]:
# Cruce anterior vs BandaV
cruce1 = df_banda.merge(cruce1, 
                        left_on='Pseudo_clave_cat', right_on='key', 
                        how='outer', indicator=True, 
                        suffixes=('_BandaV', '_Manz'))
cruce1['Tipo_cruce2'] = cruce1['_merge'].map({'both':'BandaV_&_Manzana_&_AreaH', 'left_only':'BandaV', 'right_only':'Manzanas_&_AreaH' })
cruce1.drop(columns=['_merge'], inplace=True)
# Revisamos el tipo de variable
for col in ['Val m2_BandaV','Val m2_Manz']:
    cruce1[col] = cruce1[col].astype('float')
cruce1[['Val m2_BandaV','Val m2_Manz']].fillna(0, inplace=True)
# cruce1.sort_values('Val m2_Manz',inplace=True)
cruce1.shape

(712, 28)

In [13]:
# Borramos duplicados por clave catastral y geomety
print('Shape original --> ', cruce1.shape)
cruce1.drop_duplicates(subset=['cve_cat','geometry'],inplace=True)
print('Shape final    --> ', cruce1.shape)

Shape original -->  (712, 28)
Shape final    -->  (662, 28)


In [14]:
print(f'AreaH solo tiene  {len(df_area["Manzanas"].unique())} manzanas')
print(f'BandaV solo tiene {len(df_banda["Manzanas"].unique())} manzanas')
print(f'Total de manzanas diferentes que abarcan los valores unitarios {len(df_area["Manzanas"].unique()) + len(df_banda[~df_banda["Manzanas"].isin(df_area["Manzanas"])]["Manzanas"].unique())}')
print(f'Manzanas en el cruce BandaV: {len(cruce1["Manzanas_BandaV"].unique())}')
print(f'Manzanas en el cruce AreaH: {len(cruce1["Manzanas_Manz"].unique())}')

AreaH solo tiene  317 manzanas
BandaV solo tiene 83 manzanas
Total de manzanas diferentes que abarcan los valores unitarios 324
Manzanas en el cruce BandaV: 84
Manzanas en el cruce AreaH: 310


In [15]:
# Identificamos los que no tienen valores
cruce1.loc[(cruce1['Val m2_BandaV'].fillna(0)==0) & (cruce1['Nombre Calle'].fillna(0)==0),['No_BandaV']] = 1
cruce1.loc[(cruce1['Val m2_Manz'].fillna(0)==0) & (cruce1['Tipo calle'].fillna('0')=='0'),'No_AreaH'] = 1


In [16]:
# Interpolamos para rellenar faltantes
cruce1['Val_m2_AH_I'] = cruce1['Val m2_Manz'].interpolate(method='pad') 

In [17]:
# Convertir a Geodataframe el cruce1
cruce1 = gpd.GeoDataFrame(cruce1, geometry='geometry').to_crs(3857)

## Proceso 1
Asociar un valor unitario a cada registro del modelo IA, para poder aproximar su valor catastral

In [ ]:
# Revisamos las columnas para llenar la sig celda
cruce1.columns

In [18]:
# Definimos columnas utiles
# Valores unitarios para banda de valores
col_vu_bv = 'Val m2_BandaV'
# Valores unitarios para area homogenea
col_vu_ah = 'Val_m2_AH_I'

In [19]:
# Revision de valores 0, no permitidos en ambos Val m2
cruce1[(cruce1[col_vu_ah].fillna(0)==0)&(cruce1[col_vu_bv].fillna(0)==0)].shape

(0, 31)

In [20]:
# Cargamos el GeoDataFrame del modelo de la IA
gdf_modelo_c = gpd.read_file(path_modelo_casas).to_crs(3857)
gdf_modelo_c.rename(columns={'area':'Area_Modelo'}, inplace=True)
print('Shape original -> ',gdf_modelo_c.shape)
gdf_modelo_c.drop_duplicates(subset='geometry', inplace=True)
print('Shape final -> ',gdf_modelo_c.shape)

Shape original ->  (29208, 4)
Shape final ->  (29208, 4)


In [21]:
len(gdf_modelo_c.geometry.unique())

29208

In [22]:
# Intentamos borrar la clase carros si es que existe
existe_categoria = False
try:
    s = gdf_modelo_c.shape[0]
    gdf_modelo_c.drop(index=gdf_modelo_c[gdf_modelo_c['clase_dete']=='carros'].index, inplace=True)
    print('Se borraron: ',s-gdf_modelo_c.shape[0])
    # Columnas de categorias
    casas = ['casas', 'establecimiento','multivivienda']
    terrenos = ['area_verde','terreno_baldio',]
    existe_categoria = True
except:
    print('Revise que la columnas para borrar carros exista')
    print(gdf_modelo_c.columns)

Se borraron:  0


In [23]:
# Cruce de modelo IA vs Valores Unitarios & mz
cruce2 = gpd.sjoin(gdf_modelo_c,cruce1, how='left', lsuffix='_Cruce1', rsuffix='_ModeloIA')
#cruce2['Tipo_cruce1'] = cruce2['Tipo_cruce1'].astype(str)
#cruce2['Tipo_cruce2'] = cruce2['Tipo_cruce2'].astype(str)
print('Shape original = ', cruce2.shape)
cruce2.reset_index(drop=True,inplace=True)
cruce2.dropna(subset=['cve_cat','geometry'], inplace=True)
print('Shape final    = ', cruce2.shape) 

Shape original =  (30310, 35)
Shape final    =  (28182, 35)


In [24]:
# Volvemos a revisar que los val por m2 esten llenos
cruce2[(cruce2[col_vu_bv].fillna(0)==0)&(cruce2[col_vu_ah].fillna(0)==0)]

,clase_dete,conf,Area_Modelo,geometry,index__ModeloIA,Tipo calle,Nombre Calle,Zona_BandaV,Manzanas_BandaV,COD_BandaV,...,manz,cve_cat,key,Lon,Lat,Tipo_cruce1,Tipo_cruce2,No_BandaV,No_AreaH,Val_m2_AH_I


In [25]:
# Obtener las medidas de los lados
cruce2['Medida_lx'] = cruce2['geometry'].map(lambda x: get_lados_xy(x)[0]) 
cruce2['Medida_ly'] = cruce2['geometry'].map(lambda x: get_lados_xy(x)[1]) 

In [26]:
# Cargamos 15m

# Clave de municipio
cve_mun = '040'


gdf_15m = gpd.read_file(path_info).to_crs(3857)

# Filtramos
gdf_15m = gdf_15m[gdf_15m['CVE_MUN']==cve_mun] 
gdf_15m['geometry'].tail(3)

57885    POLYGON ((-11097802.698 2134896.234, -11097764...
57886    POLYGON ((-11098135.153 2134741.547, -11098148...
57887    POLYGON ((-11098032.286 2134981.362, -11098004...
Name: geometry, dtype: geometry

In [27]:
# Cruzamos 15m con el cruce2 (cruce IA vs Manzanas)
# Cruce 
cruce3 = gpd.sjoin(cruce2, gdf_15m,
                   how='left',lsuffix='_cruce2',rsuffix='_15m')
cruce3.tail(3)

,clase_dete,conf,Area_Modelo,geometry,index__ModeloIA,Tipo calle,Nombre Calle,Zona_BandaV,Manzanas_BandaV,COD_BandaV,...,Medida_ly,index__15m,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,CVE_MZA,AMBITO,TIPOMZA
30164,casas,0.39297,80.450648,"POLYGON ((-11090094.895 2135875.313, -11090086...",427.0,NaN,NaN,NaN,NaN,NaN,...,12.3888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30165,establecimiento,0.46477,543.041872,"POLYGON ((-11090103.060 2135832.798, -11090086...",427.0,NaN,NaN,NaN,NaN,NaN,...,33.7047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30166,establecimiento,0.23680,1748.795954,"POLYGON ((-11090112.079 2135801.291, -11090079...",427.0,NaN,NaN,NaN,NaN,NaN,...,59.2306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
if cruce3[cruce3['manz'].fillna('vacio')=='vacio'].shape[0] == 0:
    print('ok')
    drop_mz = ['Zona_Manz','Manzanas_Manz','CVE_MZA']
else:
    print('revisa las filas vacias y buscales una manzana')
    drop_mz = []

ok


In [30]:
# Borrar cols inecesarias
cols_drop = ['key','Pseudo_clave_cat_BandaV','Pseudo_clave_cat_Manz',
             'Municipio_BandaV','Municipio_Manz','mun','index__15m','CVE_MUN',
             'CVE_ENT','CVE_LOC'] + drop_mz
print('Shape original: ', cruce3.shape)
cruce3.drop(columns=cols_drop,inplace=True)
print('Shape final: ', cruce3.shape)

Shape original:  (28909, 46)
Shape final:  (28909, 33)


In [ ]:
cruce3['AMBITO'].value_counts()

In [ ]:
cruce3['TIPOMZA'].value_counts()

## Proceso 1.1
Cruzamos con valores de chinchetas

In [31]:
gdf_chinchetas = gpd.read_file(path_chinchetas)
gdf_dnue = gpd.read_file(path_dnue)

In [32]:
schema_crs={'set_ch':3857,'to_ch':3857,'set_dnue':6364,'to_dnue':3857}


In [33]:
gdf_chinchetas = gdf_chinchetas.set_crs(schema_crs['set_ch'],allow_override=True)
gdf_dnue   = gdf_dnue.set_crs(schema_crs['set_dnue'],allow_override=True)
gdf_chinchetas.set_geometry('geometry',inplace=True)

In [34]:
gdf_chinchetas = cruzar_chinchetas_vs_dnue(gdf_chinchetas, gdf_dnue,
                                           municipio='Ixtapan de la Sal',
                                           show_crs=True,
                                           show_results=True)

CRS chinchetas:  EPSG:3857
CRS Dnue      :  EPSG:3857
omitio  geometry


100%|██████████| 105/105 [00:32<00:00,  3.26it/s]


omitio  geometry


100%|██████████| 299/299 [01:42<00:00,  2.91it/s]


Dnue ids encontrados    :  433
Dnue faltan por asignar :  728
Dnue total de id en dnue:  1161
Chinchetas se encontraron     :  404
Chinchetas falta por encontrar:  1412
Total chichentas              :  1816


In [ ]:
# Hacemos cruce IA vs chinchetas
cruce4 = gpd.sjoin(cruce3, gdf_chinchetas, how='inner',
          lsuffix='IA', rsuffix='ch')
# Creamos diccionario de agrupacion
cols_agg = ['Clase','tipoCenCom', 'tipo_asent', 'nombre_act', 'clee','id']
dict_agg = {}
for k in cols_agg:
    dict_agg[k] = '|'.join
    
# Hacemos todo str menos la geometry para agrupar
for col in cols_agg:
    if col == 'geometry':
        print('omitio ', col)
        continue
    cruce4[col] = cruce4[col].astype(str)
        

cruce4 = cruce4.groupby('geometry', as_index=False,sort=False).agg(dict_agg)


In [44]:
cruce4.columns

Index(['geometry', 'Clase', 'tipoCenCom', 'tipo_asent', 'nombre_act', 'clee',
       'id'],
      dtype='object')

In [49]:
# Asignamos lo encontrado al original modelo IA
for i in tqdm.tqdm(cruce4.index):
    cruce3.loc[cruce3['geometry'] == cruce4.loc[i,'geometry'], cols_agg] = cruce4.loc[i,cols_agg]


100%|██████████| 1934/1934 [29:48<00:00,  1.08it/s]


In [ ]:
from shapely.geometry import Point
c = pd.read_csv('/home/hector/Descargas/temporal/test_igecem_ch.csv',)

geometry = [eval(x) for x in c.geometry.str.replace('POINT ','Point').str.replace(' ',',')]
c = gpd.GeoDataFrame(c, geometry=geometry,crs=6364)
print(c.crs)
c.head()


In [ ]:
cruce4.shape

In [ ]:

cruce_f = gpd.sjoin(cruce4, c, how='left',
          rsuffix='IA',lsuffix='test')
cruce_f.shape

## Proceso 2
Hacer una identificacion geodesica de cada registro en el modelo IA para poder obtener factores necesarios para la aproximacion del valor catastral

In [50]:
# Hacemos un cambio de variable para mejor visualizacion
gdf = cruce3
gdf.reset_index(drop=False, inplace=True)
gdf.rename(columns={'index':'Indice_gdf'}, inplace=True)
gdf = gdf.to_crs(3857)
gdf['Esquinero|Intermedio'] = 0
gdf.shape

(28909, 41)

In [ ]:
# Matamos variables para liberar memoria
if input('Esta seguro de matar las variables?').lower() in afirmativo:
    if input('Realmente esta seguro de matar las variables?').lower() in afirmativo:
        del cruce2, cruce1, gdf_modelo_c
        print('Matamos las variables cruce2, cruce1, gdf_modelo_c, ya no hay vuelta atras')

In [51]:
# Cargamos el shape de manzanas para analisis
shape = gpd.read_file(path_manzanas)
shape=shape.to_crs("3857")
shape["centroid"]=shape.centroid
shape.shape

(661, 6)

In [52]:
# Volvemos a revisar que los val por m2 esten llenos
gdf[(gdf[col_vu_bv].fillna(0)==0)&(gdf[col_vu_ah].fillna(0)==0)]

,Indice_gdf,clase_dete,conf,Area_Modelo,geometry,index__ModeloIA,Tipo calle,Nombre Calle,Zona_BandaV,Manzanas_BandaV,...,CVE_AGEB,AMBITO,TIPOMZA,Clase,tipoCenCom,tipo_asent,nombre_act,clee,id,Esquinero|Intermedio


In [55]:
# Buscamos por distancias los mas cercanos al perimetro de la manzana

nuevos = get_polygons_nearest_perimeter(falta_fp=gdf,
                                        shape=shape,
                                        col_cve_cat='cve_cat',
                                        distancia_max=4,
                                        col_id='Indice_gdf')

100%|██████████| 631/631 [01:17<00:00,  8.19it/s]


 Faltan  28909  Claves y se revisaron  29372  Claves
Se encontraron:  5760 
 Shape final -> (7502, 2)


In [54]:
se_rompio_arriba = False
if se_rompio_arriba:
    shape.geometry = shape.geometry.map(lambda y: y.convex_hull)
    # Codigo para revisar los multipoligonos
    # shape[shape.geometry.map(lambda y: True if isinstance(y,MultiPolygon) else False)]

In [56]:
# etiquetamos los valores que son corner_polygons con el otro algoritmo
gdf.loc[gdf['Indice_gdf'].isin(np.unique(nuevos['Indice_gdf'].values)), 'Esquinero|Intermedio'] = 1

In [57]:
# Revision de conteos nuevos en el gdf
gdf['Esquinero|Intermedio'].value_counts()

Esquinero|Intermedio
0    22577
1     6332
Name: count, dtype: int64

In [58]:
# Borrar duplicados
print('Shape original ', gdf.shape)
gdf.drop_duplicates(subset=['geometry','cve_cat'],inplace=True)
print('Shape fianl ', gdf.shape)

Shape original  (28909, 41)
Shape fianl  (28182, 41)


In [59]:
# Volvemos a revisar que los val por m2 esten llenos
gdf[(gdf[col_vu_bv].fillna(0)==0)&(gdf[col_vu_ah].fillna(0)==0)]

,Indice_gdf,clase_dete,conf,Area_Modelo,geometry,index__ModeloIA,Tipo calle,Nombre Calle,Zona_BandaV,Manzanas_BandaV,...,CVE_AGEB,AMBITO,TIPOMZA,Clase,tipoCenCom,tipo_asent,nombre_act,clee,id,Esquinero|Intermedio


## Proceso 3
Ahora ya tenemos la info necesaria para el calculo de los factores lo que nos llevara al valor catastral

Todo este procedimiento a implementar es un seguimiento detallado del [manual catastral](!https://igecem.edomex.gob.mx/sites/igecem.edomex.gob.mx/files/files/ArchivosPDF/Servicios-catastrales/Manual%20Catastral%20del%20Estado%20de%20Mexico.pdf) que el IGECEM publica para el calculo del valor catastral. 

In [61]:
datos = gdf
#datos.fillna(0,inplace=True)

In [ ]:
# Matamos variables para liberar memoria
if input('Esta seguro de matar las variables?').lower() in afirmativo:
    if input('Realmente esta seguro de matar las variables?').lower() in afirmativo:
        del gdf
        print('Matamos las variables cgdf   , ya no hay vuelta atras')

In [62]:
# borrar duplicados again
print('Shape original -> ',datos.shape)
datos.drop_duplicates(subset=['geometry','cve_cat'],inplace=True)
print('Shape nuevo    -> ',datos.shape)

Shape original ->  (28182, 41)
Shape nuevo    ->  (28182, 41)


In [67]:
# Obtenemos las columnas de altura, nivel, y valores para borrarlas, ya que se hara de nuevo la clasificacion
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])
cols_otros = np.array([])
col_categoria = None
#datos.fillna(0, inplace=True)


for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
    # elif col in list(vu_const['Clase']):
    #     cols_otros = np.append(cols_otros, col)
    elif col.lower().strip().startswith('clase_dete'):
        col_categoria = col
len(cols_altura), len(cols_nivel), len(cols_valor_uni), len(cols_otros), col_categoria

(0, 0, 3, 0, 'clase_dete')

In [68]:
# Diccionarios utiles para clasificacion

dict_niveles = {
    'Niveles_Bajo':1,
    'Niveles_Medio':2,
    'Niveles_Alto':3,
    'Niveles_oficina':5,
    'Niveles_edificios':10
}

In [69]:
# Renombramos en caso de ser necesario
renombres = {'Area_H':'Area_AH'}
datos.rename(columns=renombres,inplace=True)

In [79]:
# Columnas a utilizar, pueden variar, por eso se definen aqui
col_area_modelo = 'Area_Modelo'
col_area_h = 'Area_AH'

In [71]:
# Poner niveles 
for key,val in dict_niveles.items():
    print(key,val)
    datos[key] = val

Niveles_Bajo 1
Niveles_Medio 2
Niveles_Alto 3
Niveles_oficina 5
Niveles_edificios 10


In [73]:
path_vu_construccion = './Data/Valores unitarios Construccion.xlsx'
df_construccion = pd.read_excel(path_vu_construccion)
df_construccion.tail()

,Codigo,Uso,Clase,Valor m2
60,E1,Equipamientos,HOTEL BUENO MEDIA,12365
61,E1,Equipamientos,HOTEL MUY BUENO BAJA,16297
62,E1,Equipamientos,HOTEL MUY BUENO MEDIA,18542
63,E1,Equipamientos,MERCADO BAJA,4632
64,E1,Equipamientos,MERCADO MEDIA,5807


In [74]:
# Poner valores unitarios de construccion

cols_vu_cons = np.array([])
usar = 'baja'  # baja, media, alta


for tipo in datos['Tipo'].unique():
    # print(tipo)
    tipo = str(tipo).replace(' ','')
    if tipo == '0':
        tipo = 'H4'
    df_aux = df_construccion.loc[df_construccion['Codigo']==tipo]
    for clase, val in df_aux[['Clase','Valor m2']].values:
        if str(clase).endswith(usar):
            print(clase, val)
            datos.loc[(datos['Tipo']==tipo), clase] = val
            cols_vu_cons = np.append(cols_vu_cons,clase)
            del clase, val
        else:
            continue
    del df_aux

Precaria baja 1217
Regular baja 7308
Interes social baja 5093
Comercial buena baja 10225
Economica baja 3628


In [76]:
# Obtenemos las columnas de altura, nivel, y valores
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])
cols_const = np.array([])
#datos.fillna(0, inplace=True)

for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
    # elif col in list(vu_const['Clase']):
    #     cols_const = np.append(cols_const,col)
len(cols_altura), len(cols_nivel), len(cols_valor_uni),# len(cols_const)

(0, 5, 3)

In [80]:
# Campos faltantes
datos['Frente_base'] = datos['Frente']
datos['Fondo_base']  = datos['Fondo']
datos['Area_base']   = datos[col_area_h]

if existe_categoria:
    datos.loc[datos[col_categoria].isin(casas),'grado_conservacion'] = datos.loc[datos[col_categoria].isin(casas)][col_area_h].map(lambda x:get_grado_conservacion())
else:
    datos['grado_conservacion'] = datos[col_area_h].map(lambda x:get_grado_conservacion())
datos['Area_inscrita']     = datos[col_area_modelo].map(lambda x: float(x)*random.uniform(0.65,1))
datos['Area_construccion'] = datos[col_area_modelo]*.9

In [81]:
# Columnas a utilizar, pueden variar, por eso se definen aqui
col_esquinero = 'Esquinero|Intermedio'

In [87]:
for col in ['Tipo_cruce1','Tipo_cruce2']:
    datos[col] = datos[col].astype(str)

In [100]:
datos['Area_base']  = datos['Area_base'].str.replace(',','').astype(float)

In [103]:
# Factores catastral del terreno

datos.fillna(0, inplace=True)
cols_factor_top = np.array([])

datos['Factor_posicion']      = datos[col_esquinero].fillna(0).map(lambda x: get_factor_posicion(x))
datos['Terreno_Posicion']     = datos['Factor_posicion'].map(lambda x: 'Interior' if x==0.5 else 'otro')
datos['factor_frente']        = datos.apply(lambda x: factor_frente(x['Medida_lx'], x['Terreno_Posicion']),axis=1)
datos['factor_fondo']         = datos.apply(lambda x: factor_fondo(x['Medida_ly'], x['Fondo_base'], x['Terreno_Posicion']), axis=1)
datos['Factor_Topografia']    = datos.apply(lambda x: round(random.uniform(0.9,1),5), axis=1)



datos['factor_irregularidad'] = datos.apply(lambda x: factor_irregularidad(x[col_area_modelo], x['Area_inscrita'], x['Terreno_Posicion']), axis=1)
datos['factor_area']          = datos.apply(lambda x: factor_area(x[col_area_modelo],x['Area_base'],x['Terreno_Posicion']), axis=1)
datos['factor_restriccion']   = datos.apply(lambda x: factor_restriccion(x[col_area_modelo],x[col_area_modelo]*0.8,x['Terreno_Posicion']), axis=1)

In [104]:
# Revison de valores 0 en los factores de terreno y correccion

cols_ft = {'factor_frente':0.5,'factor_fondo':0.6,'factor_irregularidad':0.5,'factor_area':0.7,'factor_restriccion':0.5}

for col, val in cols_ft.items():
    s = datos[datos[col].fillna(0)==0].shape
    
    if s[0] != 0:
        datos.loc[datos[col].fillna(0)==0, col] = val
        print(col)
    print(s)
    
for col in cols_factor_top:
    s = datos[datos[col].fillna(0)==0].shape
    if s[0] != 0:
        print(col)
    print(s)

(0, 65)
(0, 65)
(0, 65)
(0, 65)
(0, 65)


In [105]:
# Factores catastral de la construcción

cols_factor_nvl = np.array([])

if existe_categoria:
    datos.loc[datos[col_categoria].isin(casas),'factor_grado_conservacion'] = datos[datos[col_categoria].isin(casas)]['grado_conservacion'].map(lambda x: factor_grado_conservacion(x))
    datos.loc[datos[col_categoria].isin(casas),'factor_edad']               = 0.6
else:
    datos['factor_grado_conservacion'] = datos['grado_conservacion'].map(lambda x: factor_grado_conservacion(x))
    datos['factor_edad']               = 0.6


for col in cols_nivel:
    nombre = 'factor_numero_niveles_'+col
    cols_factor_nvl = np.append(cols_factor_nvl, nombre)
    if existe_categoria:
        datos.loc[datos[col_categoria].isin(casas),nombre] = datos[datos[col_categoria].isin(casas)].apply(lambda x: factor_numero_niveles(x[col],x['grado_conservacion']), axis=1)
    else:
        datos[nombre] = datos.apply(lambda x: factor_numero_niveles(x[col],x['grado_conservacion']), axis=1)

In [106]:
# Revision de valores 0 en los factores de construccion
if existe_categoria:
    print(datos[(datos[col_categoria].isin(casas))&(datos['factor_grado_conservacion'].fillna(0)==0.0)].shape)
    print(datos[(datos[col_categoria].isin(casas))&(datos['factor_edad'].fillna(0)==0.0)].shape)
    
    for col in cols_factor_nvl:
        s = datos[(datos[col_categoria].isin(casas))&(datos[col].fillna(0)==0.0)].shape
        if s[0] != 0:
            print(col)
        print(s)
else:
    print(datos[(datos['factor_grado_conservacion'].fillna(0)==0.0)].shape)
    print(datos[(datos['factor_edad'].fillna(0)==0.0)].shape)
    
    for col in cols_factor_nvl:
        s = datos[(datos[col].fillna(0)==0.0)].shape
        if s[0] != 0:
            print(col)
        print(s)


(0, 72)
(0, 72)
(0, 72)
(0, 72)
(0, 72)
(0, 72)
(0, 72)


In [107]:
# Conversion a flotantes de los valores que necesitamos

cols_using = [col_area_modelo,'factor_frente','factor_fondo', 'factor_irregularidad','factor_area',
              'Factor_posicion','factor_restriccion']
datos.fillna(0,inplace=True)
for col in cols_using:
    datos[col] = datos[col].astype(float)

![alt text](Data/Valor_cat_terreno.png) 

Formula para obtener el valor catastral del terreno

In [108]:
# Valor catastral del terreno
print('Lista de valores unitarios para terreno \n ', cols_valor_uni)
index_use = input('Que valores quiere usar para el valor por metro cuadrado(separe cada indice con una coma, empezando en 0)?: ')
if index_use.lower() in ['todos', 'all']:
    cols_vu_terr = cols_valor_uni.copy()
else:
    cols_vu_terr = cols_valor_uni[[int(x) for x in index_use.split(',')]]
cols_vc_terr = np.array([])

for col_vu in cols_vu_terr:
    name = col_vu.split('_')[-1]
    name_col = 'Valor_catastral_terreno_'+name
    datos[name_col] = datos[col_area_modelo].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_frente'].astype(float).fillna(0)*datos['factor_fondo'].fillna(0)*datos['factor_irregularidad'].fillna(0)*datos['factor_area'].fillna(0)*datos['Factor_Topografia'].fillna(0)*datos['Factor_posicion'].fillna(0)*datos['factor_restriccion'].fillna(0)
    cols_vc_terr = np.append(cols_vc_terr,name_col)

Lista de valores unitarios para terreno 
  ['Val m2_BandaV' 'Val m2_Manz' 'Val_m2_AH_I']


![alt text](Data/Valor_cat_const.png) <br>
 Formula para obtener el valor catastral de la construccion

In [109]:
# Valor catastral de la construccion
cols_vc_const = np.array([])
for col_vu in cols_vu_cons:
    i = 0
    name = col_vu.split('_')[-1]
    for col_nv in cols_factor_nvl:
        name_col = 'Valor_catastral_construccion_'+name+'_'+str(i)
        cols_vc_const = np.append(cols_vc_const,name_col)
        datos[name_col] = datos[col_area_modelo].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_edad'].astype(float).fillna(0)*datos['factor_grado_conservacion'].astype(float).fillna(0)*datos[col_nv].astype(float).fillna(0)
        i += 1

In [110]:
# Revicion del total de columnas creadas
print(f'Cols construccion -> {len(cols_vc_const)} \nCols terreno      -> {len(cols_vc_terr)} \nShape Final       -> {datos.shape}')

Cols construccion -> 25 
Cols terreno      -> 3 
Shape Final       -> (28182, 100)


In [111]:
# Volvemos a revisar que los val por m2 esten llenos
datos[(datos[col_vu_bv].fillna(0)==0)&(datos[col_vu_ah].fillna(0)==0)]

,Indice_gdf,clase_dete,conf,Area_Modelo,geometry,index__ModeloIA,Tipo calle,Nombre Calle,Zona_BandaV,Manzanas_BandaV,...,Valor_catastral_construccion_Comercial buena baja_0,Valor_catastral_construccion_Comercial buena baja_1,Valor_catastral_construccion_Comercial buena baja_2,Valor_catastral_construccion_Comercial buena baja_3,Valor_catastral_construccion_Comercial buena baja_4,Valor_catastral_construccion_Economica baja_0,Valor_catastral_construccion_Economica baja_1,Valor_catastral_construccion_Economica baja_2,Valor_catastral_construccion_Economica baja_3,Valor_catastral_construccion_Economica baja_4


#### Checkpoint opcional
 <br>Hasta este punto hemos aproximado tantos valores catastrales como el numero de columnas creadas arriba.
El siguiente proceso consume muchos recurso asi que se recomienda llevarlo a google colab

In [ ]:
nombre_file = 'New_casas_Naucalpan_proces2_v1'

In [ ]:
# Crear en CSV para no truncar el nombre de columnas
pd.DataFrame(datos).to_csv('Data/Completados/Modelo_Final/'+nombre_file+'.csv',
                           index=False,
                           encoding='utf8')

In [ ]:
# Crear el geopandas solo con las columnas utiles para reponerselas al csv
datos[['geometry', 'cve_cat','Indice_gdf']].to_file('Data/Completados/Modelo_Final/Recuperacion_geometry/'+nombre_file+'.shp',
                                         index=False)

#### Checkpoint opcional

In [144]:
# Camnibio de variable otra vez
df = datos
df.reset_index(drop=False, inplace=True)
df.rename(columns={'index':'Indice_Modelo'}, inplace=True)
df.shape

(28182, 104)

In [ ]:
# Matamos variables para liberar memoria
if input('Esta seguro de matar las variables?').lower() in afirmativo:
    if input('Realmente esta seguro de matar las variables?').lower() in afirmativo:
        del datos
        print('Matamos las variables datos   , ya no hay vuelta atras')

In [145]:
# Obtenemos las columnas de interes
cols_terreno = np.array([])
cols_const = np.array([])
for col in df.columns:
    if col.lower().find('valor_catastral_construccion')>=0:
        cols_const = np.append(cols_const, col)
    elif col.lower().find('valor_catastral_terreno')>=0:
        cols_terreno = np.append(cols_terreno, col)
len(cols_terreno), len(cols_const)

(3, 25)

In [146]:
# Volvemos a revisar que los val por m2 esten llenos
df[(df[col_vu_bv].fillna(0)==0)&(df[col_vu_ah].fillna(0)==0)]

,Indice_Modelo,Indice_Modelo,Indice_gdf,clase_dete,conf,Area_Modelo,geometry,index__ModeloIA,Tipo calle,Nombre Calle,...,Valor_catastral_construccion_Comercial buena baja_2,Valor_catastral_construccion_Comercial buena baja_3,Valor_catastral_construccion_Comercial buena baja_4,Valor_catastral_construccion_Economica baja_0,Valor_catastral_construccion_Economica baja_1,Valor_catastral_construccion_Economica baja_2,Valor_catastral_construccion_Economica baja_3,Valor_catastral_construccion_Economica baja_4,Estadisticos_terreno,Estadisticos_construccion


In [147]:
df[cols_const].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28182 entries, 0 to 28181
Data columns (total 25 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Valor_catastral_construccion_Precaria baja_0         28182 non-null  float64
 1   Valor_catastral_construccion_Precaria baja_1         28182 non-null  float64
 2   Valor_catastral_construccion_Precaria baja_2         28182 non-null  float64
 3   Valor_catastral_construccion_Precaria baja_3         28182 non-null  float64
 4   Valor_catastral_construccion_Precaria baja_4         28182 non-null  float64
 5   Valor_catastral_construccion_Regular baja_0          28182 non-null  float64
 6   Valor_catastral_construccion_Regular baja_1          28182 non-null  float64
 7   Valor_catastral_construccion_Regular baja_2          28182 non-null  float64
 8   Valor_catastral_construccion_Regular baja_3          28182 non-nul

In [148]:
# Obtenemos los estadisticos de interes, esta parte es la mas pesada
df['Estadisticos_terreno'] = df.apply(lambda x: statistics_values(x[cols_terreno]), axis=1)
df[df['Estadisticos_terreno']=='Revisar'].shape

(0, 104)

In [149]:
# Obtenemos los estadisticos de interes, esta parte es la mas pesada
df['Estadisticos_construccion'] = df.apply(lambda x: statistics_values(x[cols_const]), axis=1)
df[df['Estadisticos_construccion']=='Revisar'].shape

(17869, 104)

In [150]:
# Damos formatoa las cols Minimo, Maximo, Media, desviacion_estandar, largo, media_percentil, desviacion_std_percentiles, moda_percentiles
df = pd.concat([df,
                df['Estadisticos_terreno'].str.split('|', expand=True).rename(columns={0:   'Min_terreno',
                                                                                       1:   'Max_terreno',
                                                                                       2: 'Media_terreno',
                                                                                       3:   'STD_terreno',
                                                                                       4:   'Len_terreno',
                                                                                       5:'MediaP_terreno',
                                                                                       6:  'STDP_terreno',
                                                                                       7: 'ModaP_terreno'})], axis=1)

df = pd.concat([df,
                df['Estadisticos_construccion'].str.split('|', expand=True).rename(columns={0:   'Min_construccion',
                                                                                            1:   'Max_construccion',
                                                                                            2: 'Media_construccion',
                                                                                            3:   'STD_construccion',
                                                                                            4:   'Len_construccion',
                                                                                            5:'MediaP_construccion',
                                                                                            6:  'STDP_construccion',
                                                                                            7: 'ModaP_construccion'})], axis=1)

Calculo del intervalo de confianza suponiendo distribucion normal en los valores catastrales

In [151]:
# Valor z con nivel de significancia al 95%
z = stats.norm.ppf(.95)
z

1.6448536269514722

![alt text](./Data/Valor_cat_final.png)

Formula para obtener el valor catastral final 

![alt text](./Data/Intervalo_confianza_norm.png) Formula para intervalo de confianza

In [152]:
cols_estadisticos = ['Media_terreno','STD_terreno','Len_terreno','Media_construccion','STD_construccion','Len_construccion']

for col in cols_estadisticos:
    try:
        df[col] = df[col].astype(float)
    except:
        print(col)

In [ ]:
df.head(2)

In [ ]:
df['Media_terreno']#- (z*df['STD_terreno']/df['Len_terreno'])

In [153]:
# Limite inferior (LI) y superior (LS) de cada valor catastral
df['LI_95%_vc_terreno'] = df['Media_terreno'].astype(float).fillna(0)- (z*df['STD_terreno'].astype(float).fillna(0)/df['Len_terreno'].astype(float).fillna(0))
df['LS_95%_vc_terreno'] = df['Media_terreno'].astype(float).fillna(0)+ (z*df['STD_terreno'].astype(float).fillna(0)/df['Len_terreno'].astype(float).fillna(0)) 

df['LI_95%_vc_construccion'] = df['Media_construccion'].astype(float).fillna(0)- (z*df['STD_construccion'].astype(float).fillna(0)/df['Len_construccion'].astype(float).fillna(0))
df['LS_95%_vc_construccion'] = df['Media_construccion'].astype(float).fillna(0)+ (z*df['STD_construccion'].astype(float).fillna(0)/df['Len_construccion'].astype(float).fillna(0) )

df['LS_95%_catastral_final'] = df['LS_95%_vc_terreno'].fillna(0) + df['LS_95%_vc_construccion'].fillna(0) 
df['LI_95%_catastral_final'] = df['LI_95%_vc_terreno'].fillna(0) + df['LI_95%_vc_construccion'].fillna(0) 

In [154]:
df[[col_area_modelo,col_vu_ah, 'LI_95%_catastral_final','LS_95%_catastral_final']].head(3)

,Area_Modelo,Val_m2_AH_I,LI_95%_catastral_final,LS_95%_catastral_final
0,90.506979,17.0,698.979061,698.979061
1,63.354885,17.0,490.227746,490.227746
2,162.912562,17.0,1318.357686,1318.357686


In [155]:
cols_estadisticos = ['Media_terreno','STD_terreno','Len_terreno','Min_terreno','Max_terreno','MediaP_construccion','STDP_construccion','ModaP_construccion',
                     'Media_construccion','STD_construccion','Len_construccion','Min_construccion','Max_construccion','MediaP_construccion','STDP_construccion','ModaP_construccion',
                     'LI_95%_catastral_final','LS_95%_catastral_final']

for col in cols_estadisticos:
    try:
        if col.lower().startswith('min'):
            df[col] = df[col].str.replace('Revisar','0')
        df[col] = df[col].astype(float)
    except Exception as e:
        print(col)
        print(e)

In [156]:
# Calculo de valor catastral para estadisticos
df['minVC'] = df['Min_terreno'].astype(float).fillna(0) + df['Min_construccion'].astype(float).fillna(0)
df['maxVC'] = df['Max_construccion'].astype(float).fillna(0) + df['Max_terreno'].astype(float).fillna(0)
df['mediaVC'] = df['Media_construccion'].astype(float).fillna(0) + df['Media_terreno'].astype(float).fillna(0)
df['PmediaVC'] = df['MediaP_construccion'].astype(float).fillna(0) + df['MediaP_terreno'].astype(float).fillna(0)

In [157]:
# Solo cols de interes
df[np.append(cols_estadisticos,['Indice_gdf','geometry','cve_cat'])]

,Media_terreno,STD_terreno,Len_terreno,Min_terreno,Max_terreno,MediaP_construccion,STDP_construccion,ModaP_construccion,Media_construccion,STD_construccion,...,Min_construccion,Max_construccion,MediaP_construccion,STDP_construccion,ModaP_construccion,LI_95%_catastral_final,LS_95%_catastral_final,Indice_gdf,geometry,cve_cat
0,698.979061,0.0,1.0,698.979061,698.979061,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,698.979061,698.979061,417,"POLYGON ((-11104928.147 2135064.029, -11104918...",0600902700000000
1,490.227746,0.0,1.0,490.227746,490.227746,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,490.227746,490.227746,454,"POLYGON ((-11104715.690 2134795.864, -11104708...",0600902700000000
2,1318.357686,0.0,1.0,1318.357686,1318.357686,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,1318.357686,1318.357686,518,"POLYGON ((-11104560.229 2138343.579, -11104545...",0600705200000000
3,2439.186994,0.0,1.0,2439.186994,2439.186994,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,2439.186994,2439.186994,584,"POLYGON ((-11104511.190 2136511.342, -11104485...",0600902900000000
4,911.061862,0.0,1.0,911.061862,911.061862,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,911.061862,911.061862,585,"POLYGON ((-11104499.792 2136515.322, -11104486...",0600902900000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28177,2.675763,0.0,2.0,2.675763,2.675763,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,2.675763,2.675763,30162,"POLYGON ((-11090130.950 2135756.937, -11090120...",0600502200000000
28178,3.713825,0.0,2.0,3.713825,3.713825,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,3.713825,3.713825,30163,"POLYGON ((-11090098.783 2135837.148, -11090087...",0600502200000000
28179,1.098031,0.0,2.0,1.098031,1.098031,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,1.098031,1.098031,30164,"POLYGON ((-11090094.895 2135875.313, -11090086...",0600502200000000
28180,7.565089,0.0,2.0,7.565089,7.565089,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,7.565089,7.565089,30165,"POLYGON ((-11090103.060 2135832.798, -11090086...",0600502200000000


#### En caso de haber hecho checkpoint

In [ ]:
# En caso de haber hecho checkpoint
path_gdf =  'Data/Completados/Modelo_Final/Recuperacion_geometry/'+nombre_file+'.shp'
gdf = gpd.read_file(path_gdf)
gdf.rename(columns={'Indice_gdf':'Indice_gdf1'},inplace=True)

final = gpd.GeoDataFrame(pd.concat([gdf[['geometry', 'cve_cat','Indice_gdf1']] , df[cols_new]], axis=1), geometry='geometry')
final.tail(2)

gdf.shape, final.shape

In [ ]:
# Revision de empatamiento 
final[final['Indice_gdf']!=final['Indice_gdf1']]

In [ ]:
# Exportar el GeoDataFrame
final.to_file('/content/drive/MyDrive/Equipo_Agua/Geo/Data/Completados/Modelo_Final/Solo_VC_Final/NewCasas_Valor_Catastral_Naucalpan_Final_v1.shp',
                                                  index=False)

#### En caso de no haber hecho checkpoint

In [161]:
# Calculo del predial
cols_calc_predial = ['minVC','maxVC','mediaVC','LI_95%_catastral_final','LS_95%_catastral_final','PmediaVC']
cols_predial = []
df_predial = pd.read_csv('./Data/Predial.csv',)
df_predial['FACTOR'] = df_predial['FACTOR'].str.replace(',','.')
for col in df_predial:
    df_predial[col] = df_predial[col].astype(float)


for col in cols_calc_predial:
    name_col = 'Predial'+col
    df[name_col] = df[col].map(lambda x: calculo_predial(x,df_predial))
    cols_predial.append(name_col)

In [165]:
# Ya terminamos solo exportamos el geodataframe
ruta_and_name_save = '/home/hector/Documentos/Infis/Geo/Data/Shapes/Ixtapan_de_la_sal/Ixtapan_completo_VC_v2.shp'
if input('usar cols solo necesarias') in afirmativo:
    cols_out = np.append(cols_predial,['cve_cat','Indice_gdf','geometry']+cols_agg)
else:
    cols_out = df.columns
    
df[cols_out].to_file(ruta_and_name_save,  index=False)

In [162]:
df['comparacion_min_max'] = df['PredialminVC'] == df['PredialmaxVC']
df['comparacion_min_max'].value_counts()

comparacion_min_max
True     18170
False    10012
Name: count, dtype: int64

In [163]:
df[np.append(cols_predial,['cve_cat','Indice_gdf','geometry'])]

,PredialminVC,PredialmaxVC,PredialmediaVC,PredialLI_95%_catastral_final,PredialLS_95%_catastral_final,PredialPmediaVC,cve_cat,Indice_gdf,geometry
0,170.231031,170.231031,170.231031,170.231031,170.231031,170.231031,0600902700000000,417,"POLYGON ((-11104928.147 2135064.029, -11104918..."
1,170.161934,170.161934,170.161934,170.161934,170.161934,170.161934,0600902700000000,454,"POLYGON ((-11104715.690 2134795.864, -11104708..."
2,170.436045,170.436045,170.436045,170.436045,170.436045,170.436045,0600705200000000,518,"POLYGON ((-11104560.229 2138343.579, -11104545..."
3,170.807040,170.807040,170.807040,170.807040,170.807040,170.807040,0600902900000000,584,"POLYGON ((-11104511.190 2136511.342, -11104485..."
4,170.301230,170.301230,170.301230,170.301230,170.301230,170.301230,0600902900000000,585,"POLYGON ((-11104499.792 2136515.322, -11104486..."
...,...,...,...,...,...,...,...,...,...
28177,170.000555,170.000555,170.000555,170.000555,170.000555,170.000555,0600502200000000,30162,"POLYGON ((-11090130.950 2135756.937, -11090120..."
28178,170.000898,170.000898,170.000898,170.000898,170.000898,170.000898,0600502200000000,30163,"POLYGON ((-11090098.783 2135837.148, -11090087..."
28179,170.000032,170.000032,170.000032,170.000032,170.000032,170.000032,0600502200000000,30164,"POLYGON ((-11090094.895 2135875.313, -11090086..."
28180,170.002173,170.002173,170.002173,170.002173,170.002173,170.002173,0600502200000000,30165,"POLYGON ((-11090103.060 2135832.798, -11090086..."
